# Introdução

Neste documento iniciamos a parte de recuperacao de documentos PDF partindo do suposto que temos uma base de arquivos PDF para utilizar.

Neste preprocessamento, extraimos o texto dos pdfs para serem posteriormente processadas.


# Parte 0: Instalações

In [1]:
# %pip install PyPDF2
# %pip install unidecode

# Parte 1: Listagem dos PDFs

In [2]:
import os

In [3]:
pdf_folder_path = "./parsed_pdf"

In [4]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".txt"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [5]:
filename_list[:10]

['./parsed_pdf/documento_11422_10324.txt',
 './parsed_pdf/documento_11422_13409.txt',
 './parsed_pdf/documento_11422_14547.txt',
 './parsed_pdf/documento_11422_20921.txt',
 './parsed_pdf/documento_11422_21245.txt',
 './parsed_pdf/documento_11422_3776.txt',
 './parsed_pdf/documento_11422_4587.txt',
 './parsed_pdf/documento_11422_8763.txt',
 './parsed_pdf/documento_276189.txt',
 './parsed_pdf/documento_494080.txt']

# Parte 2: Obtenção dos Metadados da Base Minerva

In [6]:
import os
import json
import numpy as np

In [7]:
metadata_folder_path = "./metadata_documents"

In [8]:
filename_metadata_list = []
for filename in os.listdir(metadata_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_metadata_list.append(os.path.join(metadata_folder_path, filename))

filename_metadata_list = sorted(filename_metadata_list)

In [9]:
filename_metadata_list[:10] 

['./metadata_documents/documento_.json',
 './metadata_documents/documento_276189.json',
 './metadata_documents/documento_494080.json',
 './metadata_documents/documento_547235.json',
 './metadata_documents/documento_614469.json',
 './metadata_documents/documento_683353.json',
 './metadata_documents/documento_762801.json',
 './metadata_documents/documento_768599.json',
 './metadata_documents/documento_774811.json',
 './metadata_documents/documento_806362.json']

In [10]:
metadata_dict = {}

for filename in filename_metadata_list:
    with open(filename) as f:
        metadata = json.load(f)

    _id = metadata["id"]
    
    metadata_dict[_id] = metadata

# Parte 3: Recuperação do arquivo txt

In [11]:
def retrieve_document(filename):
    with open(filename, "r") as file:
        file_text = file.read()
    return file_text

## Teste

In [12]:
name = filename_list[np.random.randint(1, len(filename_list))]
print(name)

full_text = retrieve_document(name)
print(full_text[:300])

./parsed_pdf/documento_933375.txt
 
 
UNIVERSIDADE FEDERAL DO RIO DE JANEIRO  
 
 
 
SCARLATHE BEZERRA DA COSTA  
 
 
 
 
 
 
EFEITOS DA DELEÇÃO DE REGULADORES RELACIONADOS A RESPOSTA AO 
ESTRESSE OXIDATIVO NA VIRULÊNCIA DE Bacteroides fragilis   
 
 
 
 
 
 
 
 
 
 
RIO DE JANEIRO  
2022  
ii 
 
 
                     Scarlathe Bez


# Parte 4: Tratamento do Texto em Blocos

* Essa parte é importante porque os blocos precisam de um tamanho máximo de 4096 tokens para serem processados pelos modelos.
* Não podemos simplesmente cortar os blocos na metade das frases.
* Em muitos dos pdfs já estamos garantidos pelo \n duplo, que deixa um espaço entre os parágrafos.
* Mas em alguns o texto é muito fluido e não possui linhas vazias.
* Também existe uma peculiaridade do PyPDF que ele não identifica texto corrido, então as linhas ficam quebradas exatamente na mudança de linha, o que nos impede de interpretar quebra de linha como mudança de parágrafo.

**Regras de Tratamento:**
1. Quebra de blocos por saltos de linha duplicados, indicando a quebra de um parágrafo.

In [13]:
from lib.parser import TextParser

## Teste:

In [15]:
text_parser = TextParser(DEBUG_MODE=False)

In [16]:
parsed_text = text_parser.parse_full_text(full_text)

In [17]:
print(name)

./parsed_pdf/documento_933375.txt


# Parte 5: Processamento Completo

In [38]:
text_parser = TextParser()

In [39]:
final_json = {}

In [45]:
formatted_documents_path = "./formatted_documents"

In [46]:
for filename in filename_list:
    _filename = filename.split('/')[-1].split('.')[0]
    _id = "-".join(_filename.split('_')[1:])
    _id_underscore = "_".join(_filename.split('_')[1:])
    
    try:
    
        _metadata = metadata_dict[_id]

        print(_id)
        print(_metadata["title"], "\n")

        # document retrieval
        document = retrieve_document(filename)

        # parse document - break in blocks
        parsed_text = text_parser.parse_full_text(document)

        final_json[_id] = {
            **_metadata,
            "sentences": parsed_text
        }

        with open(f'{formatted_documents_path}/documento_' + _id_underscore + ".json", 'w', encoding='utf-8') as f:
            json.dump(final_json[_id], f, ensure_ascii=False)
    
    except Exception as e:
        print("Error for id", _id, "for: ", e)

Error for id 11422-10324 for:  '11422-10324'
Error for id 11422-13409 for:  '11422-13409'
Error for id 11422-14547 for:  '11422-14547'
Error for id 11422-20921 for:  '11422-20921'
Error for id 11422-21245 for:  '11422-21245'
Error for id 11422-3776 for:  '11422-3776'
Error for id 11422-4587 for:  '11422-4587'
Error for id 11422-8763 for:  '11422-8763'
276189
Psicologia, trabalho e atenção básica do SUS : compartilhando experiências em equipes NASF no Rio de Janeiro 

494080
Terapeutas cognitivo-comportamentais frente à religiosidade/espiritualidade no contexto psicoterapêutico 

547235
O teatro do contador de histórias de Tahar Ben Jelloun 

614469
Evolução do relevo, coberturas sedimentares e formação de solos em superfícies geomorfológicas : Medio Vale do Rio Paraíba do Sul (SP/RJ) 

683353
Territorialização e constituição de sujeitos políticos no orçamento participativo de Porto Alegre 

762801
A história do currículo da licenciatura em Química da UFRJ : tensões, contradições e desa